In [1]:
# imports
import os
import sys
import dvc.api
import mlflow
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# adding scripts
sys.path.append('.')
sys.path.append('..')
sys.path.insert(1, '../scripts/')
import defaults as defs
import dataCleaner as dc
import dataVisualizer as dv

cleaner = dc.dataCleaner('data preparation notebook')
visualizer = dv.dataVisualizer('data preparation notebook')

--> <Logger dataCleaner (WARNING)>
logger <Logger dataCleaner (INFO)> created at path: ../logs/cleaner_root.log
Data cleaner in action
--> <Logger dataVisualizer (WARNING)>
logger <Logger dataVisualizer (INFO)> created at path: ../logs/visualizer_root.log
Data visualizer in action


In [3]:
# pandas settings
pd.set_option('display.max_columns', 50)

version = 'v1'

# read data sets using dvc api
test_data_url = dvc.api.get_url(path = defs.test_local_path, 
                                repo = defs.repo, 
                                rev = version)

train_data_url = dvc.api.get_url(path = defs.train_local_path, 
                                repo = defs.repo, 
                                rev = version)

store_data_url = dvc.api.get_url(path = defs.store_local_path, 
                                repo = defs.repo, 
                                rev = version)
print('test data path: ' + test_data_url, '\ntrain data path: ' + train_data_url, '\nstore data path: ' + store_data_url)

test data path: gdrive://1tqW4fbN4G10FXhketTvLclHFM12_rsDo/70/2189c326ce783ff913ddd6ff984946 
train data path: gdrive://1tqW4fbN4G10FXhketTvLclHFM12_rsDo/d4/7aedd9e2d580e06a6ef7ce1732e8b6 
store data path: gdrive://1tqW4fbN4G10FXhketTvLclHFM12_rsDo/30/c9ccb5af66c21f65e7d1da967e6251


In [4]:
# reading csv files
DateCols = ['Date']
missing_values = ["n/a", "na", "undefined", '?', 'NA', 'undefined']

test_data = pd.read_csv(test_data_url, na_values=missing_values,parse_dates=DateCols)
train_data = pd.read_csv(train_data_url, na_values=missing_values, parse_dates=DateCols)
store_data = pd.read_csv(store_data_url, na_values=missing_values)

ImportError: Please install gdrivefs for access to Google Drive

# Data preparation